In [1]:
# -*- coding: utf-8 -*-
"""
Created on August 8 2021

@author: FB
"""
import matplotlib.pyplot as plt
import numpy as np

from time import sleep, monotonic

import qcodes as qc
from qcodes import Station, load_or_create_experiment, initialise_database, Measurement, load_by_run_spec, load_by_id
from qcodes.dataset.plotting import plot_dataset, plot_by_id
from qcodes.utils.dataset.doNd import do0d,do1d, do2d


In [2]:
qc.logger.start_all_logging()
station = qc.Station()

qc.initialise_or_create_database_at('F:\QCoDeSLocal\SpinQubit\database\IMEC_AL809789_D18_D3SD4_T4.db') 
exp = load_or_create_experiment(
    experiment_name= 'T4',
    sample_name="QBB16_4_6"
)

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\T4ACQ\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\T4ACQ\.qcodes\logs\210924-11852-qcodes.log


In [106]:
STATION = qc.Station(config_file='Config_24Sep2021_FB.yaml',use_monitor='True')

In [107]:
qdac=STATION.load_instrument('qdac_A')

In [6]:
dmm = STATION.load_instrument('keysight_A')

Connected to: Keysight Technologies 34465A (serial:MY54501434, firmware:A.02.09-02.37-02.09-00.49-02-01) in 0.27s


In [7]:
def Ithaco_get():   
    return dmm.volt()*1e-7*(-1)

ithaco = qc.Parameter(name='Ithaco',label='drain current',unit='A', get_cmd=Ithaco_get)

# Buffered acquistion

In [18]:
dmm_lockin = STATION.load_instrument('keysight_A_2')

Connected to: Keysight Technologies 34465A (serial:MY54501309, firmware:A.02.14-02.40-02.14-00.49-02-01) in 1.17s


In [26]:
import re
# do2d_buffered takes as channel not (e.g.) qdac.ch01.v but qdac.ch01, as I need later (in ramp_voltages_2d_buffered) the channel number (obtained by int(re.findall("\d+", <string name>)[0]))
def do2d_buffered(slow_chan, slow_vstart, slow_vend, slow_steps , fast_chan, fast_vstart, fast_vend, fast_steps ,step_length, dmm):    
    meas = Measurement()
    v1s = np.linspace(slow_vstart, slow_vend, slow_steps)
    v2s = np.linspace(fast_vstart, fast_vend, fast_steps)
    meas.register_parameter(slow_chan.v)
    meas.register_parameter(fast_chan.v)
    meas.register_custom_parameter('buffer', 'Buffer', 'V', setpoints=(slow_chan.v,fast_chan.v,))
    with meas.run() as datasaver: 
        dmm.init_measurement()   
        meas_time = qdac.ramp_voltages_2d( slow_chans=[int(re.findall("\d+", slow_chan.short_name)[0])], slow_vstart=[slow_vstart], slow_vend=[slow_vend],
                                  fast_chans=[int(re.findall("\d+", fast_chan.short_name)[0])], fast_vstart=[fast_vstart], fast_vend=[fast_vend],
                                  step_length=step_length,slow_steps = slow_steps, fast_steps = fast_steps)
                                               
        sleep(meas_time)
        data = dmm.fetch()
        datasaver.add_result((slow_chan.v, np.repeat(v1s,fast_steps)),
                             (fast_chan.v, np.tile(v2s,slow_steps)),
                             ('buffer', data))    
        dataset2D = datasaver.dataset

In [27]:
qdac.ch03.sync(1)
qdac.ch03.sync_delay(0)
qdac.ch03.sync_duration(0.001)

In [88]:
dmm_lockin.sample.source('TIM')

In [100]:
dmm_lockin.device_clear() #necessary after timeout error
dmm_lockin.reset() #can be commented
slow_steps = 51 #number of pixel along slow axis
fast_steps = 51 #number of pixel along fast axis
all_gates(2)
# qdac.ch03.v(0.4)
# qdac.ch05.v(0.4)
# sleep(1)
#dmm.timeout(5)
dmm_lockin.display.text('buffering...') # Displays the text to speed up commands of the dmm
dmm_lockin.range(10) #range in volts
dmm_lockin.aperture_mode('ON')
dmm_lockin.NPLC(0.02) #{0.2, 1, 0.06, 100, 10, 0.02}
dmm_lockin.trigger.source('EXT') #before was IMM
dmm_lockin.trigger.slope('POS')
dmm_lockin.trigger.count(1)
dmm_lockin.trigger.delay(0.00)
dmm_lockin.sample.source('TIM')

#dmm_lockin.trigger.delay(0)

dmm_lockin.sample.count(slow_steps*fast_steps)
dmm_lockin.sample.timer(0.002)
#dmm_lockin.sample.pretrigger_count(0)

do2d_buffered(qdac.ch03, 0.400, 0.600, slow_steps, qdac.ch05, 0.400, 0.600, fast_steps,0.002, dmm_lockin)

dmm_lockin.display.clear() # Returns display to its normal state


Starting experimental run with id: 880. 
